In [5]:
# This should be built on a fully commissioned ESP32
# But there's problems with getting both i2c devices 
# working from an ESP32

# So we have an MQTT version which you can get the data using
# mosquitto_sub -h 192.168.0.159 -v -t "walkingpole/#"

In [6]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [4]:
#for l in open("main.py"):
#    print(l, end="")

In [46]:
help("modules")

__main__                        select            upip_utarfile
_onewire          hashlib           socket            upysh
_thread           heapq             ssl               urandom
apa106            inisetup          struct            ure
array             io                sys               urequests
binascii          json              time              uselect
btree             machine           ubinascii         usocket
builtins          math              ucollections      ussl
cmath             micropython       uctypes           ustruct
collections       neopixel          uerrno            utime
dht               network           uhashlib          utimeq
ds18x20           ntptime           uheapq            uzlib
errno             onewire           uio               websocket
esp               os                ujson             zlib
flashbdev         random            uos
Plus any modules on the filesystem


In [39]:
from BNO055_funcs import BNO055init, BNO055poletilt, BNO055gacc

BNO055init()
print(BNO055poletilt())

BNO055 sensor SW_REV_ID: 0x11.0x3
(-0.0, -90.0)


In [44]:
p = machine.Pin(2, machine.Pin.OUT)
p.value(1)

In [40]:
for i in range(10):
    print(BNO055poletilt())


__main__          http_client_ssl   sys               urandom
_boot             http_server       time              ure
_onewire          http_server_ssl   uasyncio/__init__ urequests
_webrepl          inisetup          uasyncio/core     urllib/urequest
apa102            json              ubinascii         uselect
array             lwip              ucollections      usocket
btree             machine           uctypes           ussl
builtins          math              uerrno            ustruct
dht               micropython       uhashlib          utime
ds18x20           neopixel          uheapq            utimeq
errno             network           uio               uzlib
esp               ntptime           ujson             webrepl
example_pub_button                  onewire           umqtt/robust      webrepl_setup
example_sub_led   os                umqtt/simple      websocket
flashbdev         port_diag         uos               websocket_helper
framebuf          select            u

In [ ]:
print(BNO055poletilt())


In [34]:
# the location of the i2c pins on the firebeetle
%sendtofile i2cmodule.py
import machine
#i2c = machine.I2C(scl=machine.Pin(22), sda=machine.Pin(21))
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5), freq=100000)  #D2,D1 NodeMCU


Sent 3 lines (163 bytes) to i2cmodule.py.


In [161]:
%sendtofile --source ../essential_sensor_code/VL53L0X_funcs.py
%sendtofile --source ../essential_sensor_code/BNO055_funcs.py

Sent 46 lines (1653 bytes) to BNO055_funcs.py.


In [21]:
%sendtofile --source ../essential_sensor_code/BNO055_funcs.py

Sent 41 lines (1648 bytes) to BNO055_funcs.py.


In [5]:
%sendtofile main.py

ssid, ssidpassword = b'DoESLiverpool', b'decafbad00'
mqttbroker = "192.168.0.159"   # mqtt.local
ssid = None

from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
from BNO055_funcs import BNO055init, BNO055poletilt, BNO055gacc

from machine import Pin
import time, network, math
from umqtt.robust import MQTTClient

pled = Pin(2, Pin.OUT)

if ssid:
    staif = network.WLAN(network.STA_IF)
    staif.active(True)
    staif.connect(ssid, ssidpassword)
    while not staif.isconnected():
        pled.value(1-pled.value())
        time.sleep(0.2)
else:
    staif = network.WLAN(network.AP_IF)
    staif.active(True)
    
VL53L0Xinit()
BNO055init()

mqttclient = None

t0 = time.ticks_ms()
i = 0
while True:
    i += 1
    pled.value(i%2)
    time.sleep_ms(5)
    t1 = time.ticks_ms()
    time.sleep_ms(max(5, 50-(t1-t0)))
    t1 = time.ticks_ms()
    pled.value(1)
    
    if mqttbroker is None:
        mqttbroker = "192.168.4.%d"%((i%4)+2)
        print("trying mqttbroker", mqttbroker)
    if mqttclient is None:
        mqttclient = MQTTClient(b"walkingpole", mqttbroker)
        try:
            mqttclient.connect()
            mqttclient.publish(b"walkingpole/ip", staif.ifconfig()[0].encode())
        except OSError as e:
            print(e)
            if ssid is None:
                mqttbroker = None
            mqttclient = None
            continue
    
    foretilt, sidetilt = BNO055poletilt()
    d = VL53L0Xdist()
    gravacc = BNO055gacc()

    
    msg = b"{} {} {} {} {}".format(t1-t0, int(d*4), int(foretilt), int(sidetilt), gravacc)
    print(msg)
    mqttclient.publish(b"walkingpole/dtilt", msg)
    t0 = t1



Sent 65 lines (1634 bytes) to main.py.


In [32]:
%rebootdevice

normal repl mode not detected b'\r\nMicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: sof#9 ets_task(40100130, 3, 3fff837c, 4)\r\nWebREPL is not configured, run \'import webrepl_setup\'\r\n'
not command mode
BNO055 sensor SW_REV_ID: 0x11.0x3

 ets Jan  8 2013,rst cause:2, boot mode:(3,7)

load 0x40100000, len 31100, room 16 
tail 12
chksum 0xe3
ho 0 tail 12 room 4
load 0x3ffe8000, len 1084, room 12 
tail 0
chksum 0xc0
load 0x3ffe8440, len 3248, room 8 
tail 8
chksum 0xe1
csum 0xe1
b'\x0e\xfc\x80'

b'$\xdc\x9f|\xff\x83\x1bs{o#\x8f\x8c'b'l\xe4\x1b\x0c'b'c\xe4$\x0cb\xec\x84\x9c\xdc\xe3\xe3l'b'\x0cb\x0c\x8eds$\x0e$\x0cd\x9c\xdf|\xff\x82\x13;snc\x8e\xc4ld\x9cp\x1b{\x82#\xe4\x8c'b'c\xec\xc4\x9c\x9c\xe3\x03c\x03\x0c'b'c\xec'b'\x87l;l\x07\x84'b'l\x9c\x9f|\xfe\x83\x12{;gb\x87\x84$\x0c\x9c\x1c\x84\x0c\x8fc\x84\x0c\x0cb\xec\x84\x9c\x9c\xe3\x03\x1b\x0c'b'\x0c#\xe4lb\x07d\x8c\xe3\x02\xe4\x1b\x83n\xec\x93{\x82\xfbo|\xe4'lb'$ld`\x03\x8c\xe3r\xdbl\x8c$\x13\x84'b'l`\x03\x8c\xe2s\x93l\xe4l\x1b\xc4\x0c'b'\x0cl`\x03\x84\xe3;\x9bd\x0c\xc7\x9f\x00\x8c'b'\x0cld`\x03rl\xc7\x83;d\xc4\x9f\xe3\x8c#\x0c\x84\x8c#\xc4c'b'cs\x03\x03\xec\x8e{\x0c\x13\x1bc\xe4\xfbg\xef\x80\x1bgn\xe3\x18\x03'b'\x02d\xdb\xf3\x03\x0cd'b'\x8c'b'l\x84\x8fld\x87\x1e\xc3\xc4\x8c\x84\xe4\xc7\x07l\x80\x03o\xfc\x00\x84\x9f\xe3\xc4c\x0cd'b'l\x8f;x\x0c'b'\x02\x03\xc4\xdf\xe3\x8c#'b'$\x0c\x84c\x1c;l{ls\xf3n\x9c\xe3\x00\xc4\x0c\xe4\xc3\x13\x03cs\x0ec\xec\xe3#\xec\x8f\x9c\x1f\x02{c\xe4\x87\xc7\x1f\x13\x82dc\xecb\xec\xf3o\x9c\x9f\xc0\x80\x03c

[Timed out waiting for recognizable response]


In [14]:
from BNO055_funcs import BNO055init
BNO055init()
from i2cmodule import i2c
import ustruct, math


BNO055 sensor SW_REV_ID: 0x11.0x3


In [19]:
for i in range(10):
    accx, accy, accz = ustruct.unpack("<hhh", i2c.readfrom_mem(0x28, 0x28, 6))
    gravx, gravy, gravz = ustruct.unpack("<hhh", i2c.readfrom_mem(0x28, 0x2E, 6))
    gdot = accx*gravx + accy*gravy + accz*gravz
    print(gdot/98100, math.sqrt(gravx**2 + gravy**2 + gravz**2))
    time.sleep(1)


-0.197531 979.627
.0.872306 979.749
13.4854 979.797
1.1972 979.425
-0.465326 979.849
6.15066 980.074
.11.5832 980.28
13.8304 980.413
14.1137 980.043
5.8101 980.544


In [2]:
# Working out code below
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [3]:
from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist

In [4]:
import time
VL53L0Xinit()
for i in range(10):
    print(VL53L0Xdist())
    time.sleep(1)

35.75
36.25
37.75
43.5
38.5
.33.25
26.5
25.75
0
0


In [5]:
from i2cmodule import i2c
print(i2c.scan())

[40, 41]


In [6]:
k = i2c.readfrom_mem(0x28, 0x00, 6)
print("BNO055 sensor SW_REV_ID: %s.%s" %(hex(k[4]), hex(k[5])))


BNO055 sensor SW_REV_ID: 0x11.0x3


In [18]:
from BNO055_funcs import BNO055init, BNO055calibstat, BNO055quat, BNO055pitchrollorient
BNO055init()


BNO055 sensor SW_REV_ID: 0x11.0x3


In [ ]:
# pull rows from rotation matrix
# https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
row 1
    x = 1-2*(q2**2 + q3**2)  
    y = 2*(q1*q2 - q0*q3)
    z = 2*(q0*q2 + q1*q3)
row 2:
    x = 2*(q1*q2 + q0*q3)
    y = 1 - 2*(q1**2 + q3**2)
    z = 2*(q2*q3 - q0*q1)

    2*(q1*q3 - q0*q2)
2*(q0*q1 + q2*q3)
1 - 2*(q1**2 + q2**2)

    
    

In [46]:
print(math.atan2(0.1,1))

0.0996686


In [48]:
#BNO055init()
for i in range(10):
    q0, q1, q2, q3 = BNO055quat()
    riqsq = q0*q0 + q1*q1 + q2*q2 + q3*q3 
    iqsq = 1/max(1,riqsq)
    
    # row 3
    r31 = 2*(q1*q3 - q0*q2)*iqsq
    r32 = 2*(q0*q1 + q2*q3)*iqsq
    r33 = 1 - 2*(q1**2 + q2**2)*iqsq
    
    # r32 is down vector
    sidetilt = math.degrees(-math.atan2(r33, r32))
    foretilt = math.degrees(-math.atan2(r31, r32))
    print(foretilt, sidetilt)
    time.sleep(2)
    

90.7416 -90.1934
-4.09704 -9.60139
.-0.174708 16.2851
-1.70639 9.95241
2.67665 -16.5084
.89.798 -89.9377
90.8165 -90.2136
90.7892 -90.2064
90.7892 -90.2064
.90.7927 -90.2072


In [33]:
import math
def gg():
    w, x, y, z = BNO055quat()
    ysqr = y * y

    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + ysqr)
    X = math.degrees(math.atan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.degrees(math.asin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (ysqr + z * z)
    Z = math.degrees(math.atan2(t3, t4))
    print(X, Y, Z) # roll, pitch, yaw


In [34]:
for i in range(20):
    gg()
    time.sleep(2)

175.262 90.0 -135.375
175.211 90.0 -135.379
115.575 90.0 -115.593
.122.116 90.0 -136.48
148.77 90.0 -106.005
91.5964 -90.0 -130.071
.91.2361 90.0 -139.063
129.99 90.0 -129.894
172.098 90.0 -137.776
.166.897 90.0 -138.778
-93.193 90.0 -135.418
173.074 90.0 -139.47
.-97.5004 90.0 -135.425
105.726 90.0 -133.448
106.352 90.0 -133.274
.112.034 90.0 -137.758
97.6242 90.0 -135.096
.-137.872 90.0 -138.3
-136.725 90.0 -138.293
-136.725 90.0 -138.293
.

In [25]:
    q0, q1, q2, q3 = ustruct.unpack("<hhhh", i2c.readfrom_mem(0x28, 0x20, 8))
    riqsq = q0*q0 + q1*q1 + q2*q2 + q3*q3 
    iqsq = 1/riqsq 
    
    r02 = q0*q2*2 * iqsq
    r13 = q1*q3*2 * iqsq
    sinpitch = r13 - r02

    r01 = q0*q1*2 * iqsq
    r23 = q2*q3*2 * iqsq 
    sinroll = r23 + r01 
     
    r00 = q0*q0*2 * iqsq
    r11 = q1*q1*2 * iqsq
    r03 = q0*q3*2 * iqsq
    r12 = q1*q2*2 * iqsq
    a00=r00 - 1 + r11   
    a01=r12 + r03  
    rads = math.atan2(a00, -a01) 
    northorient = 180 - math.degrees(rads) 
    return math.degrees(math.asin(sinpitch)), math.degrees(math.asin(sinroll)), northorient


Traceback (most recent call last):
  File "<stdin>", line 1
IndentationError: unexpected indent


In [274]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to Serial /dev/ttyUSB0 baud=115200 
Ready.


In [205]:
x = [1,2]
def f(a,b,c):
    print(c)
f(9, *x)

2


In [223]:
help(network)

object <module 'network'> is of type module
  __name__ -- network
  WLAN -- <function>
  phy_mode -- <function>
  STA_IF -- 0
  AP_IF -- 1
  STAT_IDLE -- 0
  STAT_CONNECTING -- 1
  STAT_WRONG_PASSWORD -- 2
  STAT_NO_AP_FOUND -- 3
  STAT_CONNECT_FAIL -- 4
  STAT_GOT_IP -- 5
  MODE_11B -- 1
  MODE_11G -- 2
  MODE_11N -- 3
  AUTH_OPEN -- 0
  AUTH_WEP -- 1
  AUTH_WPA_PSK -- 2
  AUTH_WPA2_PSK -- 3
  AUTH_WPA_WPA2_PSK -- 4


In [224]:
import network
staif = network.WLAN(network.AP_IF)
staif.active(True)


In [242]:
help(staif)

object <WLAN> is of type WLAN
  active -- <function>
  connect -- <function>
  disconnect -- <function>
  status -- <function>
  scan -- <function>
  isconnected -- <function>
  config -- <function>
  ifconfig -- <function>


In [245]:
print(staif.config("essid"))
print(staif.status("stations"))


MicroPython-ac74e4


Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
TypeError: function takes 1 positional arguments but 2 were given


In [263]:
client = MQTTClient(b"walkingpole", '192.168.4.4')
client.connect()
client.publish(b"walkingpole/ip", staif.ifconfig()[0].encode())

In [265]:
client.publish(b"walkingpole/duung", b"dong")


In [268]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.3-8-g63826ac5c on 2017-11-01; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f237e71b198, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
